# Terminology project 

## **Spacy trainable pipe**

### AGUIAR Mathilde NIAOURI Dimitra

#### M2 NLP

# Goal and tools 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Imports and packages 

In [ ]:
!pip3 install 'spacy==3.4.3'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!python -m spacy download en_core_web_sm
!pip install spacy-transformers

## Uncomment if you want to download one of the following model
# !python -m spacy download en_core_web_md
# !python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 53.7 MB/s 
     |████████████████████████████████| 4.7 MB 63.7 MB/s 
     |████████████████████████████████| 182 kB 71.9 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 28 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import spacy
import fnmatch
import os
import re
from os import path, walk
import shutil
import glob
import pandas as pd
from spacy.scorer import Scorer
from spacy.training import Example
from spacy.tokens import Doc

Downloading the curated dataset 

In [ ]:
!wget https://raw.githubusercontent.com/MathildeAguiar/terminology_project/tree/main/data

# Data Preprocessing


blah blah blah TODO

Helpers to find files and create folders

In [ ]:
def create_folder(path):
    """Create a folder if it doesn't already exist"""
    if not os.path.isdir(path):
        os.makedirs(path)
    return


def find_files(directory, pattern='*.final'):
    """Recursively finds all files matching the pattern."""
    files = []
    for root, dirnames, filenames in walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(path.join(root, filename))
    # sort the list, to avoid mismatch in the output files
    files = sorted(files)
    return files


Adapt the format of the IOB tags to be supported by Spacy. Spacy expects tags in the following format: 
```
B-ATTRIBUTE
```
So we convert the `B` into `B-NLP` and the `I` to `I-NLP`. 

In [ ]:
def mapper(token):
  """
  Adapts the tags into a Spacy readable format
  """
  if token == "B":
    return "B-NLP"
  elif token == "I":
    return "I-NLP"
  else: 
    return token

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf 
!mkdir /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf 

In [ ]:
def retag_files():  
    """ 
    Retag the original .final files by using the Spacy tag format.
    Save the result into a .iob file.    
    """
    # First process the training files
    directory_tg_train = "/content/drive/MyDrive/Terminology_project/curated/training_data/"  
    all_training_files = find_files(directory_tg_train) 
    for f in all_training_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_train+"iob_trf/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue  
                new_file.write(new_txt)
            new_file.close()

    # 2nd process the dev files
    directory_tg_dev = "/content/drive/MyDrive/Terminology_project/curated/dev_data/" 
    all_dev_files = find_files(directory_tg_dev)
    for f in all_dev_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_dev+"iob_trf/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue  
                new_file.write(new_txt)
            new_file.close()


In [ ]:
retag_files()

### Convert the IOB files into *.spacy* binary files

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf/output 
!mkdir /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf/output  

In [ ]:
## For the training set ##
# Uncomment the line you want in function of the model you are using 
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf /content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf/output

#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_md /curated/training_data/iob /curated/training_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /curated/training_data/iob /curated/training_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /curated/training_data/iob /curated/training_data/iob/output

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf/output/2009-35-1-29-46.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf/output/2009-35-1-3-28.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf/output/2009-35-1-47-59.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_pr

In [ ]:
## For the dev set ##
# Uncomment the line you want in function of the model you are using 
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf/output

#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_md /curated/dev_data/iob /curated/dev_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /curated/dev_data/iob /curated/dev_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /curated/dev_data/iob /curated/dev_data/iob/output

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf/output/2009-35-4-597-635.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf/output/2010-36-1-1-30.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf/output/2011-37-3-421-454.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curat

# Training the pipeline

First convert the base-config into a config file and then simply using the command line `spacy train`

In [ ]:
# If the GPU is available run this cell 
!python -m spacy init fill-config /content/drive/MyDrive/Terminology_project/base_config_ner_gpu.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/output_clean_trf  

Test if there is a GPU available

In [ ]:
gpu = spacy.prefer_gpu()
print(gpu)

True


If it returns False please run this command and re run the previous cell.

In [ ]:
!python -m spacy train --gpu-id 0 /content/config.cfg --output /content/drive/MyDrive/Terminology_project/output_clean_trf --paths.train /content/drive/MyDrive/Terminology_project/curated/training_data/iob_trf/output --paths.dev /content/drive/MyDrive/Terminology_project/curated/dev_data/iob_trf/output

ℹ Saving to output directory:
/content/drive/MyDrive/Terminology_project/output_clean_trf
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-10 20:42:11,835] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-10 20:42:11,851] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-10 20:42:11,855] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-10 20:42:11,856] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

You should obtain 2 models: `model-best` and `model-base`. To test the trained Pipeline it's better to use `model-best`. 

# Testing on the test dataset

### Preprocessinf the test files 

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf

In [ ]:
def retag_test_files():
    """ 
    Retag the original .final files by using the Spacy tag format.
    Save the result into a .iob file.    
    """
    # First process the test files
    directory_tg_test = "/content/drive/MyDrive/Terminology_project/curated/test_data/"
    all_test_files = glob.glob(directory_tg_test + "/*.final")
    for f in all_test_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_test+"iob_trf/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue  
                new_file.write(new_txt)
            new_file.close()

In [ ]:
retag_test_files()

In [ ]:
path = r'/content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf'
all_files = glob.glob(path + "/*.iob")

l = []
cols = ['Token', 'Tag']

for filename in all_files:
    df = pd.read_csv(filename, names = cols,on_bad_lines='skip', sep='\s+', engine='python')
    l.append(df)

# Our tokens from test set are stored in a Dataframe
test_df = pd.concat(l, axis=0, ignore_index=True)

# drop nan values
test_df = test_df.dropna()
test_df['Token'].fillna('', inplace=True)


## Using Scorer to test on the test sentences 

In [ ]:
test_df.head()
test_df.iloc[88]
print(len(test_df['Token']))
print(len(test_df['Tag']))

2772
2772


### Using the command line

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf/output

In [ ]:
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf /content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf/output


ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf/output/2009-35-4-559-595.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf/output/2010-36-1-31-69.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf/output/2011-37-3-455-488.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_trf'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/c

In [ ]:
!python -m spacy evaluate --gpu-id 0 /content/drive/MyDrive/Terminology_project/output_clean_trf/model-best /content/drive/MyDrive/Terminology_project/curated/test_data/iob_trf/output

ℹ Using GPU: 0

================================== Results ==================================

TOK     -    
NER P   63.40
NER R   66.27
NER F   64.80
SPEED   2116 


=============================== NER (per type) ===============================

          P       R       F
NLP   63.40   66.27   64.80

